In [35]:
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [36]:
df = pd.read_csv('data/foodWebDataBase.csv', index_col=0)

/tmp/ipykernel_9986/1255689621.py:1: DtypeWarning: Columns (2,8,12,13,14,15,16,17,22,26,27,28,29,30,31,35,36,39,40,41,42,43,44,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/foodWebDataBase.csv', index_col=0)


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222151 entries, 1 to 222151
Data columns (total 49 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   link.citation               222151 non-null  object 
 1   link.methodology            65980 non-null   object 
 2   interaction.type            220230 non-null  object 
 3   interaction.dimensionality  203794 non-null  object 
 4   interaction.classification  221196 non-null  object 
 5   con.taxonomy                222151 non-null  object 
 6   con.taxonomy.level          214109 non-null  object 
 7   con.common                  63016 non-null   object 
 8   con.lifestage               138613 non-null  object 
 9   con.metabolic.type          216677 non-null  object 
 10  con.movement.type           222099 non-null  object 
 11  con.size.citation           47025 non-null   object 
 12  con.size.method             39003 non-null   object 
 13  con.length.min.cm. 

In [38]:
total_linhas = df.shape[0]
contagem_nao_nulos = df.count()
porcentagem_nao_nulos = (contagem_nao_nulos / total_linhas) * 100

resumo_nulos = pd.DataFrame({
    'Tipo de Dado': df.dtypes,
    'Contagem Não-Nula': contagem_nao_nulos,
    'Total de Linhas': total_linhas,
    'Porcentagem Não-Nula (%)': porcentagem_nao_nulos.round(2)
})

resumo_ordenado = resumo_nulos.sort_values(
    by='Porcentagem Não-Nula (%)',
    ascending=True
)

print("Resumo da Porcentagem de Valores Não-Nulos por Coluna (Similar ao df.info()):")
print("------------------------------------------------------------------------")
print(resumo_ordenado)


print("\nPorcentagem de Não-Nulos (Print Direto):")
print(porcentagem_nao_nulos.round(2).astype(str) + ' %')

Resumo da Porcentagem de Valores Não-Nulos por Coluna (Similar ao df.info()):
------------------------------------------------------------------------
                           Tipo de Dado  Contagem Não-Nula  Total de Linhas  \
Unnamed: 48                      object                364           222151   
Unnamed: 49                      object                510           222151   
Unnamed: 46                      object                659           222151   
Unnamed: 47                      object               5908           222151   
notes                            object              30822           222151   
res.size.method                  object              38657           222151   
con.size.method                  object              39003           222151   
res.size.citation                object              44554           222151   
con.size.citation                object              47025           222151   
res.common                       object              59189 

In [39]:
df.describe()

,con.mass.mean.g.,con.mass.max.g.,res.mass.mean.g.,res.mass.max.g.
count,2.221510e+05,222151.000000,2.221510e+05,222151.000000
mean,1.192526e+04,-997.793576,3.139098e+02,-998.189502
std,4.295109e+05,65.242275,1.124244e+05,36.957645
min,-9.990000e+02,-999.000000,-9.990000e+02,-999.000000
25%,2.040570e-04,-999.000000,1.550000e-06,-999.000000
50%,3.593440e-03,-999.000000,2.202710e-04,-999.000000
75%,1.000000e-01,-999.000000,2.500000e-03,-999.000000
max,5.239535e+07,2568.732374,5.239535e+07,3500.000000


In [40]:
df['depth'].unique()

array([-999.0, 2.5, 600, '600', '46.251182',
       'Intertidal study sites in Chile : Curaumilla',
       'Intertidal study sites in Chile: El Quisco',
       'Intertidal study sites in Chile: Las Cruces (ECIM)',
       'Intertidal study sites in Chile: Los Molles', '-999', 13.0, 0.5,
       0.1, 100.0, '100', 'Florida mangrove islands', 'lakes'],
      dtype=object)

In [41]:
target_values = ['NA', '-999']
occurrences = df.astype(str).isin(target_values).sum(axis=0)
porcentagem_ocorrencias = (occurrences / total_linhas) * 100
print(f"Contagem de ocorrências de {target_values} por coluna:")

resumo_ocorrencias = pd.DataFrame({
    'Contagem de Ocorrências': occurrences,
    'Porcentagem de Ocorrências (%)': porcentagem_ocorrencias.round(2)
})

resumo_ordenado = resumo_ocorrencias.sort_values(
    by='Porcentagem de Ocorrências (%)',
    ascending=False
)

print(resumo_ordenado)

Contagem de ocorrências de ['NA', '-999'] por coluna:
                            Contagem de Ocorrências  \
con.mass.min.g.                              221641   
res.mass.min.g.                              221641   
con.length.max.cm.                           221277   
res.length.max.cm.                           221277   
con.length.min.cm.                           199497   
res.length.min.cm.                           199497   
con.length.mean.cm.                          172871   
altitude                                     168350   
res.length.mean.cm.                          156552   
depth                                        149838   
res.taxonomy.level                             5906   
sampling.time                                  5545   
sampling.start.year                            5398   
geographic.location                            4565   
res.taxonomy                                   1009   
ecosystem.type                                  510   
res.lifesta

In [42]:
df_cleaned = df[['interaction.type','con.taxonomy', 'con.taxonomy.level', 'res.taxonomy', 'res.taxonomy.level', 'geographic.location', 'con.movement.type', 'res.movement.type']]

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222151 entries, 1 to 222151
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   interaction.type     220230 non-null  object
 1   con.taxonomy         222151 non-null  object
 2   con.taxonomy.level   214109 non-null  object
 3   res.taxonomy         222151 non-null  object
 4   res.taxonomy.level   192582 non-null  object
 5   geographic.location  222151 non-null  object
 6   con.movement.type    222099 non-null  object
 7   res.movement.type    217048 non-null  object
dtypes: object(8)
memory usage: 15.3+ MB


In [43]:
df['interaction.type'].unique()

array(['predacious', 'parasitic', 'herbivorous', 'detritivorous',
       'bacterivorous', 'fungivorous', '  gut/stomach analysis',
       ' direct observation', nan, 'parasitoid', ' gut/stomach analysis'],
      dtype=object)

In [44]:
df_a = df_cleaned[df['interaction.type'] == 'fungivorous']
len(df_a)

3838

In [45]:
df_cleaned = df_cleaned[(df_cleaned['con.taxonomy.level'] == 'species') & (df_cleaned['res.taxonomy.level'] == 'species')].dropna()
bracket_pattern = r'\s*[\(\[\{].*?[\)\]\}]'
multi_space_pattern = r'\s+'
cols_to_clean = ['con.taxonomy', 'res.taxonomy']

for col in cols_to_clean:
    # Step 1: Remove (juv), {m}, [id], etc.
    df_cleaned[col] = df_cleaned[col].str.replace(bracket_pattern, '', regex=True)
    
    # Step 2: Normalize spacing (turn "Ursus   Maritimus" into "Ursus Maritimus")
    df_cleaned[col] = df_cleaned[col].str.replace(multi_space_pattern, ' ', regex=True)
    
    # Step 3: Strip leading/trailing whitespace just in case
    df_cleaned[col] = df_cleaned[col].str.strip()

    # Step 4: (Highly Recommended) Standardize Case
    # This ensures "Ursus maritimus" and "ursus Maritimus" become the same node.
    df_cleaned[col] = df_cleaned[col].str.title()
len(df_cleaned)

99602

In [46]:
df_cleaned.describe()

,interaction.type,con.taxonomy,con.taxonomy.level,res.taxonomy,res.taxonomy.level,geographic.location,con.movement.type,res.movement.type
count,99602,99602,99602,99602,99602,99602,99602,99602
unique,6,1790,1,3311,1,25,5,8
top,predacious,Veigaia Nemorensis,species,Veigaia Nemorensis,species,Germany,walking,walking
freq,72871,1561,99602,1063,99602,29458,57104,44341


In [47]:
df_cleaned['res.movement.type'].unique()

array(['walking', 'flying', 'swimming', 'sessile', 'floating', 'other',
       'invertebrate', 'endotherm vertebrate'], dtype=object)

In [48]:
counts_df = df_cleaned['geographic.location'].value_counts().reset_index()
counts_df.columns = ['geographic.location', 'count'] # Optional: Rename columns for clarity
print(counts_df)

                                geographic.location  count
0                                           Germany  29458
1                                               USA  18223
2             Eastern Weddell Sea Shelf, Antarctica  15618
3                                    United Kingdom  11149
4                                          Portugal   4636
5                      Ireland/West Cork/Skibbereen   4508
6             Florida Key islands, Florida Bay, USA   2583
7                       Switzerland, Lake Neuchatel   2400
8   Puerto Rico-Virgin Islands (PRVI) shelf complex   1989
9      Tributaries of the Taieri River, New Zealand   1482
10                                      Brasil (CE)   1459
11                   Santa Barbara, California, USA   1138
12                        Arctic Shelf Area, Arctic   1065
13                                          Iceland    707
14                                           Canada    706
15                                       Mozambique    6

In [49]:
df_cleaned.sort_values('con.taxonomy')
df_cleaned.to_csv('data/foodWebCleaned.csv')

In [50]:
df_edges = df_cleaned.dropna(subset=['con.taxonomy', 'res.taxonomy'])
edges = list(zip(df_edges['con.taxonomy'], df_edges['res.taxonomy']))
g = ig.Graph.TupleList(edges, directed=True)

vlist = g.vs()
list_sorted = sorted(vlist, key=lambda v: v['name'])
for v in vlist:
    print(v.index, v['name'], len(v.incident()))

0 Emberiza Schoeniclus 158
1 Stilicus Rufipes 105
2 Trogophloeus Corticinus 0
3 Aphtona Lutescens 0
4 Pisaura Mirabilis 94
5 Cloeon Simile 0
6 Platystethus Nodifrons 0
7 Anacaena Limbata 0
8 Gabrius Fermoralis 30
9 Marpissa Radiata 45
10 Chaetocnema Confusa 0
11 Lesteva Sicula 68
12 Pelochares Versicolor 0
13 Diplocampa Assimile 6
14 Tetartopeus Terminatum 30
15 Argiope Bruennichi 36
16 Ilione Lineata 0
17 Trypoxylon Attenuatum 0
18 Pteronemobius Heydenii 0
19 Propylaea Quatuordecimpunctata 2
20 Ligidium Hypnorum 0
21 Chartoscirta Cincta 0
22 Dryops Auriculatus 0
23 Limnia Paludicola 0
24 Pachybrachius Sp 0
25 Paederus Riparius 118
26 Zora Spinimana 78
27 Falco Subbuteo 37
28 Conocephalus Dorsalis 43
29 Pirata Latitans 6
30 Pirata Tenuitarsis 12
31 Tetanocera Ferruginea 0
32 Cicadella Viridis 0
33 Limnobaris Dolorosa 0
34 Ilione Albiseta 0
35 Pardosa Prativaga 19
36 Tibellus Maritimus 47
37 Coenagrion Pulchellum 33
38 Somatochlora Flavomaculata 62
39 Chrysotus Cilipes 0
40 Teuchophorus